# Simulating gametes, simulating gamete marker sequencing, and back-inferring recombination probabilities in two-locus examples:

### imports

In [31]:
import scipy.stats as st
import numpy as np
import scipy.integrate as integrate
import toyplot
import h5py

from tqdm.notebook import tqdm

import poolparty

## define a continuous pdf to sample from.

### make a cool-shaped recombination map with all values greater than 0.

In [2]:
toyplot.scatterplot(np.linspace(0,1,1000), 
                   (1+1*np.cos(21*np.linspace(0,1,num=1000)+np.sin(60*np.linspace(0,1,num=1000)))), # cool equation here
                   height=300,
                   width=500);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc15f74bda6ef4c2e8184d5d7628a20ff"> <g style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0" class="toyplot-Datum" transform="translat

### does it integrate to 1 (the pdf underlying our recombination map should...)?

In [3]:
integrate.quad(lambda x: (1+1*np.cos(21*x+np.sin(60*x))), 0, 1)[0]

1.01519239365713

### nope (no surprise there), so let's find a scalar for it. 

In [2]:
# now define scaling by that previous number:
scalar = 1 / integrate.quad(lambda x: (1+1*np.cos(21*x+np.sin(60*x))), 0, 1)[0] # one over previous line

# now look at new result (should equal 1!)
integrate.quad(lambda x: (1+1*np.cos(21*x+np.sin(60*x))) * scalar, 0, 1)[0]

1.0

### now look at it scaled:

In [5]:
toyplot.scatterplot(np.linspace(0,1,1000), 
                   (1+1*np.cos(21*np.linspace(0,1,num=1000)+np.sin(60*np.linspace(0,1,num=1000))))*scalar, # cool equation here
                   height=300,
                   width=500);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tb4b8fdb3e31844a58238e14790027776"> <g style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0" class="toyplot-Datum" transform="translate(403.95395

### let's set this equation as our pdf.

In [6]:
class my_pdf(st.rv_continuous):
    def _pdf(self,x):
        expression = (1+1*np.cos(21*x+np.sin(60*x))) * scalar # scaling by the multiplier to bring max draw down to 1
        return (expression)  # Normalized over its range, in this case [0,1]

In [7]:
samps=my_pdf(a=0,b=1).rvs(size=10000)
np.max(samps) # what is the maximum x-coordinate that we sample

0.999984960703585

In [8]:
toyplot.bars(np.histogram(samps,100),
             height=300,
             width=500);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="300.0px" viewBox="0 0 500.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5bea97e12f0e489190783a9c3bbf3877"> 0.0 0.5 1.0 0 100 200

## Now let's make an array of gametes!

In [2]:
scalar = 1 / 1.01519239365713
class my_pdf(st.rv_continuous):
    def _pdf(self,x):
        expression = (1+1*np.cos(21*x+np.sin(60*x))) * scalar # scaling by the multiplier to bring max draw down to 1
        return (expression)  # Normalized over its range, in this case [0,1]

In [4]:
sim_obj = poolparty.Sim_Gamete_Sequencing(
                      directory='/pinky/patrick/poolparty_sims/sims/20gpa_192nali_100loci_384e3reads/',
                      pdf=my_pdf(a=0,b=1),
                      num_gams = 20*192,
                      gpa = 20,
                      nali=192,
                      ncutsites=100,
                      num_reads = (20*192*100) * 1, # amount for 1x coverage at each locus, times coverage amount
                 )

In [5]:
sim_obj.sim_gametes_and_sequencing()

Simulating gametes...
Sequencing gametes...



Demultiplexing...



Assigning haplotypes to loci -- this might take a while...


### Now let's look at the results:

In [3]:
haps = h5py.File('/pinky/patrick/poolparty_sims/sims/20gpa_192nali_100loci_384e3reads/haplotypes.hdf5','r')

### Where are the loci?

In [4]:
np.array(haps['loci_locs'])

array([0.00126073, 0.00264172, 0.00842809, 0.02879147, 0.04005606,
       0.0498708 , 0.06594589, 0.07509186, 0.09810929, 0.1089214 ,
       0.11244465, 0.11474154, 0.12205093, 0.12224916, 0.13026551,
       0.13434045, 0.13788821, 0.1726131 , 0.18219757, 0.18867927,
       0.19757591, 0.20459387, 0.25314558, 0.25518063, 0.26065908,
       0.2624378 , 0.27402616, 0.28620837, 0.28878921, 0.29130871,
       0.30857452, 0.30880176, 0.31525409, 0.32180081, 0.33170064,
       0.33803027, 0.34263371, 0.34673722, 0.36371038, 0.37851447,
       0.38856866, 0.39289131, 0.39862379, 0.40335419, 0.44014485,
       0.44587809, 0.4497688 , 0.45515456, 0.48920557, 0.50119812,
       0.51092223, 0.5178863 , 0.5251882 , 0.53867166, 0.55150709,
       0.55694322, 0.55715569, 0.57025554, 0.57918315, 0.59316441,
       0.59920802, 0.6010815 , 0.60651407, 0.61463967, 0.63067209,
       0.64257243, 0.64742318, 0.68275911, 0.68906486, 0.70223933,
       0.71156388, 0.72783617, 0.72905328, 0.73165613, 0.73874

Look at the distribution...

In [5]:
toyplot.scatterplot(np.array(haps['loci_locs']),np.repeat(0,len(np.array(haps['loci_locs']))),height=300,width=1000);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="300.0px" viewBox="0 0 1000.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8e53a01cbf6e40239c82da4118879ed3"> 0.0 0.5 1.0 -0.5 0.0 0.5

### With our data and a two-locus test, can we distinguish a high-recomb region from a low-recomb region?

### Identify a low recomb region:
Let's look at the low-recombination region between locus 17 and locus 20.

In [6]:
canvas = toyplot.Canvas(width=1000, height=300)
axes = canvas.cartesian()
mark = axes.scatterplot(np.linspace(0,1,1000), 
                   (1+1*np.cos(21*np.linspace(0,1,num=1000)+np.sin(60*np.linspace(0,1,num=1000))))*scalar)
mark = axes.scatterplot(np.array(haps['loci_locs']),np.repeat(1,len(np.array(haps['loci_locs']))))

mark = axes.plot(a=np.array([0,2]),b=np.array([haps['loci_locs'][17],haps['loci_locs'][17]]),
                 color='red',
                 along='y');

mark = axes.plot(a=np.array([0,2]),b=np.array([haps['loci_locs'][20],haps['loci_locs'][20]]),
                 color='red',
                 along='y');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="300.0px" viewBox="0 0 1000.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td25cb664df114ecf9646de37bbdc28b9">

Because we have the true pdf and we know the exact locations of these two loci, we can integrate the pdf to determine the true probability of a recombination event landing between them:

In [7]:
integrate.quad(lambda x: (1+1*np.cos(21*x+np.sin(60*x)))*scalar, # pdf equation
               haps['loci_locs'][17], # start point
               haps['loci_locs'][20], # stop point
              )[0]

0.0008523724182626158

This tells us that, given our recombination map, a given recombination event has a TINY chance of landing between locus 17 and locus 20 (between the two vertical lines in the above plot).

### Identify a high recomb region:
Let's look at the high-recombination region between locus 53 and locus 57.

In [8]:
canvas = toyplot.Canvas(width=1000, height=300)
axes = canvas.cartesian()
mark = axes.scatterplot(np.linspace(0,1,1000), 
                   (1+1*np.cos(21*np.linspace(0,1,num=1000)+np.sin(60*np.linspace(0,1,num=1000))))*scalar)
mark = axes.scatterplot(np.array(haps['loci_locs']),np.repeat(1,len(np.array(haps['loci_locs']))))

mark = axes.plot(a=np.array([0,2]),b=np.array([haps['loci_locs'][53],haps['loci_locs'][53]]),
                 color='red',
                 along='y');

mark = axes.plot(a=np.array([0,2]),b=np.array([haps['loci_locs'][57],haps['loci_locs'][57]]),
                 color='red',
                 along='y');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1000.0px" height="300.0px" viewBox="0 0 1000.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t309d572b3d7a471eb59653fae07a7e56">

Because we have the true pdf and we know the exact locations of these two loci, we can integrate the pdf to determine the true probability of a recombination event landing between them:

In [9]:
integrate.quad(lambda x: (1+1*np.cos(21*x+np.sin(60*x)))*scalar, # pdf equation
               haps['loci_locs'][53], # start point
               haps['loci_locs'][57], # stop point
              )[0]

0.061802552311189744

How much more probable is a recombination event here vs. in the other window?

In [10]:
0.061802552311189744 / 0.0008523724182626158 # high recomb region prob / low recomb region prob

72.50651356969225

This tells us that, given our recombination map, a given recombination event has a MUCH better chance (72.5 times higher!) of landing between locus 53 and locus 57 (between the two vertical lines in the above plot).

# Back-inferring recomb probability

### First define the known parameters:

In [4]:
# the number of total aliquots sequenced -- used to calculate expected coverage per locus
nalis = 192

# number of gametes per aliquot
ali_size = 20

# number of loci per gamete that can be sequenced
nloci = 100

# sequencing effort. It's really just the number of random draws drom the pooled aliquots
nreads = 384000

# from the poisson process of selecting a single locus from all
# pooled loci, which is a tiny probability that happens many times.
exp_cov_per_locus = nreads/nalis/nloci
exp_cov_per_locus 

20.0

In [5]:
# prob of a second crossover for each gamete. This is something defined by me. 
p_1 = 0.2

## Low recomb region example:

### Start with the two low-recomb loci, and look at the haplotype numbers at those loci across all 192 aliquots.

In [6]:
# what what is locus Alpha, how many hap1s do we observe there?
locusnumAlpha = 17 # locus location (out of 100)
n_1_obs_set = np.array([np.sum(np.array(haps[str(i)][str(locusnumAlpha)])) for i in range(192)])

# what what is locus Beta, how many hap1s do we observe there?
locusnumBeta = 20 # locus location (out of 100)
n_2_obs_set = np.array([np.sum(np.array(haps[str(i)][str(locusnumBeta)])) for i in range(192)])

# hold the haplotype data at these loci across all aliquots
n1n2 = np.vstack([n_1_obs_set,n_2_obs_set]).T

### Remind ourselves of the "real" probability of recombination between these two loci.

In [7]:
# what is the true recombination rate between these two loci, used in simulation?
# this is the number we are trying to recover
p_2 = integrate.quad(lambda x: (1+1*np.cos(21*x+np.sin(60*x)))*scalar, haps['loci_locs'][locusnumAlpha], haps['loci_locs'][locusnumBeta])[0]
p_2

0.0008523724182626158

### Now, pretending we don't know the real answer, cycle through possible p_2 values, recording the likelihood of each.

In [8]:
# calculate the likelihood of observing n_1_obs hap1 reads at locus 1 and n_2_obs hap1 reads at locus 2
full_set = []
# test across values of p, finding which one yields the highest likelihood.
for p_2 in tqdm(np.linspace(0,1,num=101)):
    set1 = []
    for n_1_obs, n_2_obs in n1n2:
        total_lik = 0
        for n_1 in range(ali_size+1):
            for n_2 in range(ali_size+1):
                # prob of getting n_1 = 0
                p_n_1 = poolparty.utils.binomial(k = n_1, n = ali_size, p = 0.5) # .5 prob of each alpha haplo being 1

                # prob of getting n_1_obs if n_1 = 0?
                p_n_1_obs = poolparty.utils.poisson_pmf(k=n_1_obs, lam = (n_1/ali_size)*exp_cov_per_locus)

                # prob of n_2 being zero if n_1 is zero?
                p_n_2 = poolparty.utils.calc_n_2_prob(n_1,n_2,p_1,p_2,ali_size)

                # prob of getting n_2_obs if n_2 = 0?
                p_n_2_obs = poolparty.utils.poisson_pmf(k=n_2_obs, lam = (n_2/ali_size)*exp_cov_per_locus)

                # total prob
                if 0 not in [p_n_1,p_n_1_obs,p_n_2,p_n_2_obs]:
                    total_lik += np.exp( np.log(p_n_1)+np.log(p_n_1_obs)+np.log(p_n_2)+np.log(p_n_2_obs) )
                else:
                    total_lik += 0
        set1.append(np.log(total_lik))
    full_set.append(np.array(set1))
    
prob_sums = []
for i in full_set:
    prob_sums.append(np.sum(i[(~np.isnan(i)).astype(int) + (~np.isinf(i)).astype(int) == 2]))

canvas = toyplot.Canvas(width=300, height=300)
axes = canvas.cartesian()
mark = axes.scatterplot(np.linspace(0,1,num=101), np.array(prob_sums))

100%|██████████| 101/101 [08:29<00:00,  5.04s/it]


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t00ac0a0a6a19403d88220f15f3444c40"> 0.0 0.5 1.0 -1055 -1050 -1045 -1040 -1035

### Predicted recombination probability (argmax of curve above):

In [9]:
# this should recover something close to our simulated p_2 value
np.linspace(0,1,num=101)[np.argmax(prob_sums)]

0.05

This is too high compared to our "real" recombination probability of ~0.0009. It might be improved with help from many pairwise comparisons...

## High recomb example:

### Now let's look at the two loci bounding a high-recombining region. 
### First, isolate the haplotype numbers at those loci across all 192 aliquots:

In [10]:
# what what is locus Alpha, how many hap1s do we observe there?
locusnumAlpha = 53 # locus location (out of 100)
n_1_obs_set = np.array([np.sum(np.array(haps[str(i)][str(locusnumAlpha)])) for i in range(192)])

# what what is locus Beta, how many hap1s do we observe there?
locusnumBeta = 57 # locus location (out of 100)
n_2_obs_set = np.array([np.sum(np.array(haps[str(i)][str(locusnumBeta)])) for i in range(192)])

# hold the haplotype data at these loci across all aliquots
n1n2 = np.vstack([n_1_obs_set,n_2_obs_set]).T

### Remind ourselves of the "real" probability of recombination between these two loci.

In [11]:
# what is the true recombination rate between these two loci, used in simulation?
# this is the number we are trying to recover
p_2 = integrate.quad(lambda x: (1+1*np.cos(21*x+np.sin(60*x)))*scalar, haps['loci_locs'][locusnumAlpha], haps['loci_locs'][locusnumBeta])[0]
p_2

0.061802552311189744

### Now, pretending we don't know the real answer, cycle through possible p_2 values, recording the likelihood of each.

In [12]:
# calculate the likelihood of observing n_1_obs hap1 reads at locus 1 and n_2_obs hap1 reads at locus 2
full_set = []
# test across values of p, finding which one yields the highest likelihood.
for p_2 in tqdm(np.linspace(0,1,num=101)):
    set1 = []
    for n_1_obs, n_2_obs in n1n2:
        total_lik = 0
        for n_1 in range(ali_size+1):
            for n_2 in range(ali_size+1):
                # prob of getting n_1 = 0
                p_n_1 = poolparty.utils.binomial(k = n_1, n = ali_size, p = 0.5) # .5 prob of each alpha haplo being 1

                # prob of getting n_1_obs if n_1 = 0?
                p_n_1_obs = poolparty.utils.poisson_pmf(k=n_1_obs, lam = (n_1/ali_size)*exp_cov_per_locus)

                # prob of n_2 being zero if n_1 is zero?
                p_n_2 = poolparty.utils.calc_n_2_prob(n_1,n_2,p_1,p_2,ali_size)

                # prob of getting n_2_obs if n_2 = 0?
                p_n_2_obs = poolparty.utils.poisson_pmf(k=n_2_obs, lam = (n_2/ali_size)*exp_cov_per_locus)

                # total prob
                if 0 not in [p_n_1,p_n_1_obs,p_n_2,p_n_2_obs]:
                    total_lik += np.exp( np.log(p_n_1)+np.log(p_n_1_obs)+np.log(p_n_2)+np.log(p_n_2_obs) )
                else:
                    total_lik += 0
        set1.append(np.log(total_lik))
    full_set.append(np.array(set1))

    
prob_sums = []
for i in full_set:
    prob_sums.append(np.sum(i[(~np.isnan(i)).astype(int) + (~np.isinf(i)).astype(int) == 2]))

canvas = toyplot.Canvas(width=300, height=300)
axes = canvas.cartesian()
mark = axes.scatterplot(np.linspace(0,1,num=101), np.array(prob_sums))

100%|██████████| 101/101 [08:21<00:00,  4.97s/it]


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8a3928743cc1443b96aa6f2ece2350cc"> 0.0 0.5 1.0 -1060 -1050 -1040

### Predicted recombination probability (argmax of curve above):

In [13]:
# this should recover something close to our simulated p_2 value
np.linspace(0,1,num=101)[np.argmax(prob_sums)]

0.04

Despite being the higher-recomb-probability region ( true probability is 0.061802552311189744), we've inferred a lower number than in the low-recomb region... More argument for many pairwise comparisons. 